### 1. Mineria de Datos sobre Redes Sociales y Sitios Web

#### 1.1. Tomando datos de la red social Twitter, al menos 200 tweets, realice un script que recolecte, procese y vizualice los datos con las siguientes restricciones:

##### a)Los tres trends más RT del momento.

In [1]:
#Cargamos los paquetes necesarios para entrar a la API de Twitter
import twitter
import json
from prettytable import PrettyTable
from collections import Counter
import pandas as pd

keys = pd.read_json('keys.json')['KEYS']
CONSUMER_KEY = keys['CONSUMER_KEY']
CONSUMER_SECRET = keys['CONSUMER_SECRET']
OAUTH_TOKEN = keys['OAUTH_TOKEN']
OAUTH_TOKEN_SECRET = keys['OAUTH_TOKEN_SECRET']

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

#### Seteamos los WOEID para obtener los resultados 

In [2]:
SPAIN_WOE_ID = 23424950
spain_trends = twitter_api.trends.place(_id=SPAIN_WOE_ID)
hashtags = spain_trends[0]['trends']
trends = map(lambda i: hashtags[i]['name'], range(len(hashtags)))[:3]
pt = PrettyTable()
pt.add_column('Spain Trends', trends)
print pt

+--------------------------+
|       Spain Trends       |
+--------------------------+
|     Felipe González      |
| #VotarNaranjaEsVotarAzul |
|           Luke           |
+--------------------------+


#### b )   Listar nombres de usuarios que publicaron con los hashtag de los trends del punto anterior.

In [3]:
# Obtenemos los tweets
CANT = 10
in_spain = set(trends)

tweets = []
for i in range(len(in_spain)):
    results = twitter_api.search.tweets(q=in_spain.pop() , count=CANT)
    tweets += results['statuses']

# Obtenemos los nombres de los users
screen_name = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
screen_name = list(set(screen_name)) #sin repetidos
pt = PrettyTable()
pt.add_column('Users', screen_name )

#mostramos los resultados en una tabla
print pt

+-----------------+
|      Users      |
+-----------------+
|    marquesqt    |
|     djanguez    |
|   protestona1   |
|    zambustrio   |
|    colladov61   |
|     jmpugae     |
| PennacchiGiulia |
|     anagsr1     |
|  albertomilanes |
|   Guaje_Dulce   |
|   Jaysaysheyyy  |
|   GonZaLoMBGE   |
|     nueydora    |
|   PabloMNaran   |
|  Nancy_Lescano  |
|  elgordomarian  |
|   NelebaBalene  |
|   SebasPodemos  |
|   rubyhoran_1D  |
|     caloi007    |
|      SM5sos     |
|  Yourmycutiepie |
|     0utcastx    |
|  abbraccjamiash |
|   joannacantos  |
|    jcalvodiaz   |
|    IKaliayev    |
|  rainbowhemmo_  |
|    Manolimon    |
+-----------------+


#### c) Cuales son las cinco palabras más utilizadas en los tweets del primer item.

In [4]:
text = map( lambda i: tweets[i]['text'], range(len(tweets)))
words = [ word for twt in text for word in twt.split() ]
c = Counter(words)
#print c.most_common()

pt = PrettyTable(field_names=['Words', 'Count'])
map(lambda r: pt.add_row(r), c.most_common()[:5])
print pt

+--------------------------+-------+
|          Words           | Count |
+--------------------------+-------+
|            RT            |   23  |
|            de            |   10  |
|          Felipe          |   10  |
| #VotarNaranjaEsVotarAzul |   9   |
|           que            |   7   |
+--------------------------+-------+


#### d) Listar los primeros 10 usuarios con mayor cantidad de seguidores

In [5]:
lista_users_followers = map( lambda i: (tweets[i]['user']['screen_name'],tweets[i]['user']['followers_count']), range(len(tweets)))

lista_users_followers = sorted(lista_users_followers, key=lambda x: x[1], reverse=True)[:10]
users = map((lambda x: x[0]), lista_users_followers)
followers = map((lambda x: x[1]), lista_users_followers)


pt = PrettyTable()
pt.add_column('screen_name', users)
pt.add_column('followers_count', followers)
print pt

+----------------+-----------------+
|  screen_name   | followers_count |
+----------------+-----------------+
|  protestona1   |      11333      |
|  Guaje_Dulce   |       6853      |
|   marquesqt    |       3939      |
|  rubyhoran_1D  |       3556      |
| rainbowhemmo_  |       3111      |
|  joannacantos  |       2549      |
| abbraccjamiash |       2382      |
|     SM5sos     |       1823      |
|   jcalvodiaz   |       1754      |
| Nancy_Lescano  |       1510      |
+----------------+-----------------+


#### e) Listar la ubicación (o en su defecto time-zone ) del tweet.

In [6]:
users = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
location_or_tz = map( lambda i: tweets[i]['user']['location'] or tweets[i]['user']['time_zone'], range(len(tweets)))

pt = PrettyTable()
pt.add_column('User', users)
pt.add_column('Location or TZ', location_or_tz)
print pt

+-----------------+----------------------------+
|       User      |       Location or TZ       |
+-----------------+----------------------------+
|  Nancy_Lescano  |    Arrecife, Lanzarote     |
|    colladov61   |            None            |
|    zambustrio   |     Lleida, CAT, ESP.      |
|   SebasPodemos  |         Ljubljana          |
|   protestona1   |          Sevilla           |
|   NelebaBalene  |         Carpe Diem         |
|     anagsr1     |         Casablanca         |
|    jcalvodiaz   |           Madrid           |
|   Guaje_Dulce   |       Madrid-Oviedo        |
|   SebasPodemos  |         Ljubljana          |
|   joannacantos  |        Philippines         |
| PennacchiGiulia |      Sydney,Australia      |
|   Jaysaysheyyy  |      A teenage vessel      |
|      SM5sos     |            None            |
|  rainbowhemmo_  | Pacific Time (US & Canada) |
|  Yourmycutiepie |            None            |
|   rubyhoran_1D  |  Los Angeles California    |
|  abbraccjamiash | 

#### f) Listar los cinco tweets más populares.


In [7]:
texto_y_rt = map( lambda i: (tweets[i]['text'], tweets[i]['retweet_count'], tweets[i]['user']['id']), range(len(tweets)))

texto_y_rt = sorted(texto_y_rt, key=lambda x: x[1], reverse=True)[:5]
popular_tw = texto_y_rt[0]
texto = map((lambda x: x[0]), texto_y_rt)
rts = map((lambda x: x[1]), texto_y_rt)


pt = PrettyTable()
pt.add_column('Tweet', texto)
pt.max_width['Tweet'] = 80
pt.add_column('Retweets', rts)
pt.align = 'l'
print pt

+----------------------------------------------------------------------------------+----------+
| Tweet                                                                            | Retweets |
+----------------------------------------------------------------------------------+----------+
| RT @itsmexamm: กูนี่ไม่ได้เฮิร์ทอะไรกับแฟนดอม 5SOS เลย แค่เห็นคนที่ Luke               | 68       |
| จูงมือแล้วนึกถึง Briana ขึ้นมาแค่นั้นเอง ปิดหน้า แต่ยิ้ม สเต็ปเดีย…                      |          |
| RT @most5sosfanfics: i will legit only like this relationship if luke is happy   | 54       |
| that's all that matters to me                                                    |          |
| RT @NODOticias: Felipe Gonzalez, no haga el ridículo..                           | 43       |
| Pinochet no encarcelaba a los "opositores" los aseninaba.                        |          |
| Fueron 40.000. http://t.co…                                                      |          |
| RT @ForretsGump: Se ve q

#### g) Listar a los seguidores del autor del tweet más popular

In [8]:
import tweepy
auth_tweepy = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth_tweepy.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth_tweepy)
user = api.get_user(popular_tw[2])
title = "%s's followers:" % (user.screen_name)
followers = map((lambda f: f.screen_name), user.followers(count=100))

pt = PrettyTable()
pt.add_column(title, followers)
print pt

+-----------------------+
| nueydora's followers: |
+-----------------------+
|      FamousNature     |
|     DiversifyBand     |
|      HotOrNot_USA     |
|      chibisovar2      |
|         psonar        |
|        kppppkk        |
|     CaltotheRescue    |
|     Waran23574231     |
|      NateMaingard     |
|       chi2chill_      |
|       jarukitmm       |
|      playabit162      |
|       BillBillTW      |
|       SwanseaApp      |
|     Patarapron_009    |
|     Kimberelysieve    |
|     rootbeertommo     |
|       gimenaa60       |
|      minimeanmean     |
|      NichayananF      |
|    MercuryGirlshop    |
|      thatcha1636      |
|      pann2844786      |
|       BoatBiBi_       |
|        JaYLhIsM       |
|        rdnwang        |
|      paynenaconda     |
|        lwtspxrk       |
|       KR_Kurenai      |
|     E_flowerTataR     |
|      Nancy_sition     |
|     Matt_Dominguez    |
|        miwmaeww       |
|    JonRoweOfficial    |
|        ppheats        |
|       tunt

### 2. Tomando datos de la red Social Google+, recolecte, procese y vizualice los datos con las siguientes restricciones:

#### a) Obtener las tres últimas actividades de X usuario.

In [9]:
import apiclient.discovery
import httplib2

#Accedemos a la api de google+ con la clave correspondiente
API_KEY = keys['G+'] 
service = apiclient.discovery.build('plus', 'v1', 
                                    http=httplib2.Http(), 
                                    developerKey=API_KEY)

people_feed = service.people().search(query='Linus Torvalds').execute()
#print json.dumps(people_feed['items'], indent=1)

#El usuario elegido en este caso es: Linus Torvalds
id_linus = people_feed['items'][0]['id']
activity_feed = service.activities().list(
  userId= id_linus,
  collection='public',
  maxResults='3'
).execute()

print json.dumps(activity_feed, indent=1)

KeyError: 'G+'

#### b) Analizar el texto de una de estas actividades. (Palabras utilizadas, cantidad de veces, etc)

In [ ]:
feeds_content = []
feeds = activity_feed['items'] 
for i in range(len(feeds)):
    feeds_content.append(feeds[i]['object']['content'])
print(feeds_content)

In [ ]:
words = [ word for twt in feeds_content for word in twt.split() ]
c = Counter(words)

pt = PrettyTable(field_names=['Words', 'Count'])
map(lambda r: pt.add_row(r), c.most_common()[:15])
print pt

#### c) Visualizar el avatar de tres individuos a seleccionar.

In [ ]:
from IPython.display import Image, display

def get_user_avatar(username,width, height):
    people_feed = service.people().search(query=username).execute()
    if len(people_feed['items']) > 0:    
        url = (people_feed['items'][0]['image']['url']).split('?')[0] 
    else:
        url = 'http://placehold.it/{}x{}'.format(width, height)
    return Image(url = url, width = width, height = height )

users = ['Linus Torvalds', 'Guido Van Rossum', 'Larry Page' ]
imgs = map((lambda user: get_user_avatar(user, 150, 150)), users)
_ = map((lambda img: display(img)), imgs)

### 3. Sacando información de Microformatos en la Web

#### a) Obtener y visualizar coordenadas geográficas.

Obtengo la Página de el edificio Cirque Apartments Dallas, TX

In [10]:
import urllib2
from bs4 import BeautifulSoup

page = urllib2.urlopen('http://www.umovefree.com/Apartment/cirque-apartments-dallas-tx/')
soup = BeautifulSoup(page, "lxml")

In [ ]:
#soup.prettify

Parseo y Obtengo latitud y longitud

In [11]:
longitude = soup.find('meta', itemprop='longitude')['content']
latitude = soup.find('meta', itemprop='latitude')['content']
zoom=12
print "Las Coordenadas son: " + latitude,longitude,zoom

Las Coordenadas son: 32.7900778 -96.8086546 12


In [12]:
url = "http://maps.google.com/maps?z=12&t=m&q=loc:%s+%s" % (latitude,longitude)
print url

http://maps.google.com/maps?z=12&t=m&q=loc:32.7900778+-96.8086546


In [13]:
from IPython.display import IFrame
from IPython.core.display import display

#Muestro el resultado
display(IFrame(url + "&output=embed", '425px', '350px'))

#### b) Listar calendario de eventos.

Obtengo listado de Eventos Populares en la ciudad de Los Angeles

In [14]:
import feedparser
fp = feedparser.parse('http://www.experiencela.com/rss/feeds/xlaevents.aspx?id=whatshot')

In [15]:
fechas = map(lambda e: fp.entries[e].datetime, range(len(fp.entries)))
resumenes = map(lambda e: fp.entries[e].title, range(len(fp.entries)))
#print fechas

In [16]:
from prettytable import PrettyTable
pt = PrettyTable()
pt.add_column('Fecha', fechas)
pt.add_column('Resumen', resumenes)
print pt

+-----------------------------------------+-------------------------------------------------+
|                  Fecha                  |                     Resumen                     |
+-----------------------------------------+-------------------------------------------------+
|  September 18, 2015; 6:00 PM - 7:00 PM  |    Sunset Ocean Swim Workout at Hermosa Beach   |
|       September 19, 2015; 2:30 PM       |              He-Man and She-Ra Day!             |
|  September 19, 2015; 2:00 PM - 4:30 PM  |  L.A. Conservancy's Modern Skyline Walking Tour |
| September 20, 2015; 10:30 AM - 12:00 PM |      Skirball Playdate: A Recipe for Family     |
|  September 23, 2015; 7:00 PM - 10:00 PM |              Rooftop Concert Series             |
|  September 24, 2015; 5:00 PM - 9:00 PM  |                    Beach Eats                   |
|  September 25, 2015; 6:00 PM - 8:00 PM  |         Robotic Cartography - Scale Up!         |
|  September 25, 2015; 6:00 PM - 7:00 PM  | Sunset Ocean Swi